In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, ridge_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [27]:
df = pd.read_csv('../data/final_dataset.csv')
df.head()

,NAME,ISO_TIME,LAT,LON,filename,year_sine,day_sine,moon_phase,vel_ir_x_1,vel_ir_y_1,...,bt_ir_4,bt_wv_4,bt_ir_5,bt_wv_5,bt_ir_6,bt_wv_6,bt_ir_7,bt_wv_7,bt_ir_8,bt_wv_8
0,GONU,31-05-2007 12:00,14.0,71.7,20070531120000,0.530730,-3.216245e-16,0.107425,3.890548,2.667469,...,34.061371,34.056407,34.039674,34.063149,34.166181,34.103520,34.183684,34.045542,34.168418,33.956680
1,GONU,31-05-2007 15:00,14.1,71.7,20070531150000,0.530730,-7.071068e-01,0.082893,-0.831821,-3.809046,...,34.049581,33.892392,34.056023,33.919890,34.149742,33.923957,34.022169,33.912115,33.948635,33.785965
2,GONU,31-05-2007 18:00,14.2,71.5,20070531180000,0.530730,-1.000000e+00,0.058265,2.851204,-8.899987,...,33.947206,33.541651,33.939028,33.572453,33.991307,33.595008,33.918697,33.587346,33.829673,33.583516
3,GONU,31-05-2007 21:00,14.3,71.3,20070531210000,0.530730,-7.071068e-01,0.033555,-7.948708,-10.741302,...,34.026834,33.370249,33.999726,33.351386,33.919564,33.319302,33.686716,33.278280,33.450694,33.230209
4,GONU,01-06-2007 00:00,14.4,70.9,20070601000000,0.516062,0.000000e+00,0.008777,-4.840156,-6.098961,...,33.910205,33.115717,33.870687,33.086828,33.724634,33.064566,33.653499,33.064689,33.552093,33.053363


In [28]:
df.columns

Index(['NAME', 'ISO_TIME', 'LAT', 'LON', 'filename', 'year_sine', 'day_sine',
       'moon_phase', 'vel_ir_x_1', 'vel_ir_y_1', 'vel_ir_x_2', 'vel_ir_y_2',
       'vel_ir_x_3', 'vel_ir_y_3', 'vel_ir_x_4', 'vel_ir_y_4', 'vel_ir_x_5',
       'vel_ir_y_5', 'vel_ir_x_6', 'vel_ir_y_6', 'vel_ir_x_7', 'vel_ir_y_7',
       'vel_ir_x_8', 'vel_ir_y_8', 'vel_wv_x_1', 'vel_wv_y_1', 'vel_wv_x_2',
       'vel_wv_y_2', 'vel_wv_x_3', 'vel_wv_y_3', 'vel_wv_x_4', 'vel_wv_y_4',
       'vel_wv_x_5', 'vel_wv_y_5', 'vel_wv_x_6', 'vel_wv_y_6', 'vel_wv_x_7',
       'vel_wv_y_7', 'vel_wv_x_8', 'vel_wv_y_8', 'bt_ir_1', 'bt_wv_1',
       'bt_ir_2', 'bt_wv_2', 'bt_ir_3', 'bt_wv_3', 'bt_ir_4', 'bt_wv_4',
       'bt_ir_5', 'bt_wv_5', 'bt_ir_6', 'bt_wv_6', 'bt_ir_7', 'bt_wv_7',
       'bt_ir_8', 'bt_wv_8'],
      dtype='object')

In [29]:
df = df[df.columns.drop(list(df.filter(regex='bt_')) + ['ISO_TIME', 'filename', 'year_sine', 'day_sine', 'moon_phase'])]
df.columns

Index(['NAME', 'LAT', 'LON', 'vel_ir_x_1', 'vel_ir_y_1', 'vel_ir_x_2',
       'vel_ir_y_2', 'vel_ir_x_3', 'vel_ir_y_3', 'vel_ir_x_4', 'vel_ir_y_4',
       'vel_ir_x_5', 'vel_ir_y_5', 'vel_ir_x_6', 'vel_ir_y_6', 'vel_ir_x_7',
       'vel_ir_y_7', 'vel_ir_x_8', 'vel_ir_y_8', 'vel_wv_x_1', 'vel_wv_y_1',
       'vel_wv_x_2', 'vel_wv_y_2', 'vel_wv_x_3', 'vel_wv_y_3', 'vel_wv_x_4',
       'vel_wv_y_4', 'vel_wv_x_5', 'vel_wv_y_5', 'vel_wv_x_6', 'vel_wv_y_6',
       'vel_wv_x_7', 'vel_wv_y_7', 'vel_wv_x_8', 'vel_wv_y_8'],
      dtype='object')

$\frac{dx}{dt} = \alpha F(U_{wv}) + (1-\alpha) G(U_{ir}) - W_{\beta} - W_{\beta} e^\gamma \exp(H(\frac{dU_{wv}}{dV_{wv}}, \frac{dU_{ir}}{dV_{ir}}))$ 

$\frac{dy}{dt} = \alpha F(U_{wv}) + (1-\alpha) G(U_{ir}) + W_{\beta} + W_{\beta} e^\gamma \exp(H(\frac{dU_{wv}}{dV_{wv}}, \frac{dU_{ir}}{dV_{ir}}))$ 

In [ ]:
for wavelength in ['ir', 'wv']:
    for i in range(1, 8):
        col1 = f'vel_{wavelength}_x_{i+1}'
        col2 = f'vel_{wavelength}_x_{i}'
        col3 = f'vel_{wavelength}_y_{i+1}'
        col4 = f'vel_{wavelength}_y_{i}'

        diff_col = f'sheer_{wavelength}_{i}'
        df[diff_col] = (df[col1] - df[col2]) / (df[col3] - df[col4])
df

,NAME,LAT,LON,vel_ir_x_1,vel_ir_y_1,vel_ir_x_2,vel_ir_y_2,vel_ir_x_3,vel_ir_y_3,vel_ir_x_4,...,sheer_ir_5,sheer_ir_6,sheer_ir_7,sheer_wv_1,sheer_wv_2,sheer_wv_3,sheer_wv_4,sheer_wv_5,sheer_wv_6,sheer_wv_7
0,GONU,14.0,71.7,3.890548,2.667469,1.375605,2.726160,0.114576,3.508197,-0.714657,...,1.341151,24.751665,0.504574,0.336062,-0.024071,-0.780754,-6.737869,0.202689,-1.675607,3.464838
1,GONU,14.1,71.7,-0.831821,-3.809046,1.022472,-3.501873,3.028279,-2.987043,3.160049,...,-0.101705,2.373325,-0.583739,6.836544,0.224516,-0.940391,0.604409,2.532158,0.403926,1.936144
2,GONU,14.2,71.5,2.851204,-8.899987,2.350226,-8.965180,3.385078,-8.493334,4.968107,...,-10.613989,-0.438468,-5.590788,-9.166326,-5.519516,-2.084255,-1.132437,4.793183,0.893625,1.658916
3,GONU,14.3,71.3,-7.948708,-10.741302,-5.482547,-8.791681,-3.099443,-8.362788,-2.276884,...,1.422533,-3.255419,0.653730,1.247584,5.007346,-2.953322,1.096910,0.687741,0.322910,-0.625802
4,GONU,14.4,70.9,-4.840156,-6.098961,-6.545629,-4.830266,-6.423888,-5.117756,-4.778643,...,0.415881,-0.880767,11.152531,-3.150703,-64.437240,1.249835,0.431939,0.171808,-0.171629,-0.509661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,VARDAH,13.0,72.9,-10.789840,-6.845653,-10.282471,-6.922923,-9.334971,-6.088308,-9.804493,...,0.402676,0.152361,1.227868,5.128915,1.534435,0.234166,0.452391,0.821053,0.646362,0.290920
1186,VARDAH,12.9,72.6,-9.874391,-7.047628,-9.243372,-9.236704,-7.913551,-10.928621,-6.759402,...,0.187107,25.400747,0.418517,-0.688562,-14.679469,-2.153624,-0.548828,0.950246,1.878855,0.929300
1187,VARDAH,12.8,72.2,-10.502004,-10.897576,-10.682403,-9.806686,-12.336168,-5.659580,-11.168266,...,0.970205,1.573481,1.816061,-0.777931,-0.649177,0.411991,-3.901151,-1.608430,8.699608,-4.576780
1188,VARDAH,12.6,71.8,-12.028784,-10.222869,-10.676384,-9.622954,-11.477905,-9.290735,-14.349704,...,0.391968,0.470396,-3.273235,4.713704,3.778941,-6.835735,-5.426861,0.124630,0.933934,-40.436182


In [31]:
dx = None
dy = None
flag = False

for cyclone in df.NAME.unique():
    lon = df[df.NAME == cyclone].LON.diff()
    lat = df[df.NAME == cyclone].LAT.diff()

    if not flag and not dx:
        dy = lat
        dx = lon
        flag = True

    else:
        dy = pd.concat([dy, lat], ignore_index=True)
        dx = pd.concat([dx, lon], ignore_index=True)

df['dx'] = dx
df['dy'] = dy

df.head()

,NAME,LAT,LON,vel_ir_x_1,vel_ir_y_1,vel_ir_x_2,vel_ir_y_2,vel_ir_x_3,vel_ir_y_3,vel_ir_x_4,...,sheer_ir_7,sheer_wv_1,sheer_wv_2,sheer_wv_3,sheer_wv_4,sheer_wv_5,sheer_wv_6,sheer_wv_7,dx,dy
0,GONU,14.0,71.7,3.890548,2.667469,1.375605,2.726160,0.114576,3.508197,-0.714657,...,0.504574,0.336062,-0.024071,-0.780754,-6.737869,0.202689,-1.675607,3.464838,NaN,NaN
1,GONU,14.1,71.7,-0.831821,-3.809046,1.022472,-3.501873,3.028279,-2.987043,3.160049,...,-0.583739,6.836544,0.224516,-0.940391,0.604409,2.532158,0.403926,1.936144,0.0,0.1
2,GONU,14.2,71.5,2.851204,-8.899987,2.350226,-8.965180,3.385078,-8.493334,4.968107,...,-5.590788,-9.166326,-5.519516,-2.084255,-1.132437,4.793183,0.893625,1.658916,-0.2,0.1
3,GONU,14.3,71.3,-7.948708,-10.741302,-5.482547,-8.791681,-3.099443,-8.362788,-2.276884,...,0.653730,1.247584,5.007346,-2.953322,1.096910,0.687741,0.322910,-0.625802,-0.2,0.1
4,GONU,14.4,70.9,-4.840156,-6.098961,-6.545629,-4.830266,-6.423888,-5.117756,-4.778643,...,11.152531,-3.150703,-64.437240,1.249835,0.431939,0.171808,-0.171629,-0.509661,-0.4,0.1


In [32]:
df.dropna(inplace=True)
df = df[df.columns.drop(['NAME', 'LAT', 'LON'])]
df

,vel_ir_x_1,vel_ir_y_1,vel_ir_x_2,vel_ir_y_2,vel_ir_x_3,vel_ir_y_3,vel_ir_x_4,vel_ir_y_4,vel_ir_x_5,vel_ir_y_5,...,sheer_ir_7,sheer_wv_1,sheer_wv_2,sheer_wv_3,sheer_wv_4,sheer_wv_5,sheer_wv_6,sheer_wv_7,dx,dy
1,-0.831821,-3.809046,1.022472,-3.501873,3.028279,-2.987043,3.160049,-2.455386,2.929457,-0.905439,...,-0.583739,6.836544,0.224516,-0.940391,0.604409,2.532158,0.403926,1.936144,0.0,0.1
2,2.851204,-8.899987,2.350226,-8.965180,3.385078,-8.493334,4.968107,-8.534008,4.965324,-10.893552,...,-5.590788,-9.166326,-5.519516,-2.084255,-1.132437,4.793183,0.893625,1.658916,-0.2,0.1
3,-7.948708,-10.741302,-5.482547,-8.791681,-3.099443,-8.362788,-2.276884,-7.090789,-0.560798,-6.176479,...,0.653730,1.247584,5.007346,-2.953322,1.096910,0.687741,0.322910,-0.625802,-0.2,0.1
4,-4.840156,-6.098961,-6.545629,-4.830266,-6.423888,-5.117756,-4.778643,-4.305073,-3.231138,-4.410740,...,11.152531,-3.150703,-64.437240,1.249835,0.431939,0.171808,-0.171629,-0.509661,-0.4,0.1
5,0.703742,-3.247344,0.782994,-2.789725,1.559402,-2.951125,2.364887,-2.062328,3.782991,-1.710193,...,-0.221575,1.118565,10.082778,-17.121923,0.744842,-0.267720,0.678294,1.131803,-0.4,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,-10.789840,-6.845653,-10.282471,-6.922923,-9.334971,-6.088308,-9.804493,-5.848684,-8.897868,-5.344221,...,1.227868,5.128915,1.534435,0.234166,0.452391,0.821053,0.646362,0.290920,-0.3,-0.1
1186,-9.874391,-7.047628,-9.243372,-9.236704,-7.913551,-10.928621,-6.759402,-10.533500,-6.562933,-10.278507,...,0.418517,-0.688562,-14.679469,-2.153624,-0.548828,0.950246,1.878855,0.929300,-0.3,-0.1
1187,-10.502004,-10.897576,-10.682403,-9.806686,-12.336168,-5.659580,-11.168266,-5.475892,-12.442271,-4.659163,...,1.816061,-0.777931,-0.649177,0.411991,-3.901151,-1.608430,8.699608,-4.576780,-0.4,-0.1
1188,-12.028784,-10.222869,-10.676384,-9.622954,-11.477905,-9.290735,-14.349704,-9.021186,-15.890336,-10.890754,...,-3.273235,4.713704,3.778941,-6.835735,-5.426861,0.124630,0.933934,-40.436182,-0.4,-0.2


# Solver

In [44]:
X = df[df.columns.drop(['dy', 'dx'])].to_numpy()
X_hat = df[['dx', 'dy']].to_numpy()

X.shape, X_hat.shape

((1164, 46), (1164, 2))